# Prediction Insights

## Overview

Howso Engine enables powerful predictions with complete attribution and detailed explanations to make learning from
and debugging your data and predictions as easy as possible. For more information on predictions with Howso Engine,
check out the [predictions user guide](https://docs.howso.com/user_guide/basics/predictions.html).

In [1]:
from pprint import pprint

import pandas as pd
from pmlb import fetch_data
import plotly.graph_objects as go
import plotly.express as px

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Setup

The [basic workflow guide](https://docs.howso.com/user_guide/basics/basic_workflow.html) goes into more specifics about the individual details of this section. This recipe will focus more on the insights.

### Load Data and Create Trainee

In [2]:
df = fetch_data("iris", local_cache_dir="../../data")
train_data = df.iloc[:-30]
new_data = df[~df.index.isin(train_data.index)]
features = infer_feature_attributes(train_data)

t = Trainee(features=features)

### Train, Analyze, and React

In [3]:
t.train(train_data)
t.analyze()
t.react_into_features(similarity_conviction=True)

reaction = t.react(
    contexts=new_data,
    context_features=["sepal-length", "sepal-width", "petal-length", "petal-width"],
    action_features=["target"],
    details={
        "influential_cases": True,
        "similarity_conviction": True,
        "feature_contributions": True,
        "feature_residuals": True,
        "robust_influences": True,
        "robust_residuals": True,
        "local_case_feature_residual_convictions": True,
        "categorical_action_probabilities": True,
    }
)

Note that, unlike in the basic workflow guide, we include several `details` in the react call. These are what will enable the insights that we're going to get after the predictions are made.

For more information, see [the API documentation for `Trainee.react()`](https://docs.howso.com/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

### Inspect the Predictions

Howso Engine has high accuracy even on small datasets.

In [4]:
train_data = train_data.astype({"target": str})
predicted_data = pd.concat([new_data.reset_index(drop=True).drop(columns="target"), reaction["action"]], axis=1)
predicted_data = predicted_data.astype({"target": str})

cmap = px.colors.qualitative.D3
fig = go.Figure()
for label, group in train_data.groupby("target"):
    fig.add_trace(go.Scatter(
        x=group["petal-length"],
        y=group["petal-width"],
        mode="markers",
        name=label,
        marker=dict(color=cmap[int(label)], opacity=0.75),
        legendgroup="trained",
        legendgrouptitle_text="Trained Target",
    ))

for label, group in predicted_data.groupby("target"):
    fig.add_trace(go.Scatter(
        x=group["petal-length"],
        y=group["petal-width"],
        mode="markers",
        marker=dict(size=12, symbol="star", color=cmap[int(label)], opacity=0.75),
        name=label,
        legendgroup="predicted",
        legendgrouptitle_text="Predicted Target",
        hovertext=group.index,
    ))

fig.update_layout(
    xaxis_title="Petal Length",
    yaxis_title="Petal Width",
    width=1250,
    title="Trained and Predicted Values"
)
fig.show()

For categorical action features, the prediction can be further understood with the `categorical_action_probabilities` detail.  This information
can highlight cases that are on the border of two classes, like some of the above points are.  The closer a case gets to a class border, the
more mixed the categorical action probabilities may get.

In [5]:
pprint(reaction["details"]["categorical_action_probabilities"], compact=True)

[{'target': {'2': 1}}, {'target': {'2': 1}},
 {'target': {'1': 0.12158151678205559, '2': 0.8784184832179444}},
 {'target': {'2': 1}},
 {'target': {'1': 0.3679089709331638, '2': 0.6320910290668362}},
 {'target': {'1': 1}}, {'target': {'1': 1}}, {'target': {'1': 1}},
 {'target': {'1': 1}}, {'target': {'1': 1}}, {'target': {'0': 1}},
 {'target': {'0': 1}}, {'target': {'0': 1}}, {'target': {'0': 1}},
 {'target': {'0': 1}}, {'target': {'2': 1}}, {'target': {'2': 1}},
 {'target': {'2': 1}}, {'target': {'2': 1}}, {'target': {'2': 1}},
 {'target': {'1': 1}}, {'target': {'1': 1}},
 {'target': {'1': 0.878424395035966, '2': 0.12157560496403398}},
 {'target': {'1': 0.8769119107056446, '2': 0.12308808929435557}},
 {'target': {'1': 0.38014383251032097, '2': 0.619856167489679}},
 {'target': {'0': 1}}, {'target': {'0': 1}}, {'target': {'0': 1}},
 {'target': {'0': 1}}, {'target': {'0': 1}}]


### Insight 1: Which Cases Contributed?

Howso provides complete attribution for any and all predictions, showing exactly which cases influenced each prediction.  This can be used to understand or debug predictions.  For instance,
we can inspect the influential cases for one of the cases (case `24`) that was on the decision boundary in the plot above.  We can see that there is a mix of target values and no cases stand
out in terms of influence weight:

In [6]:
inf_cases_24 = pd.DataFrame(reaction["details"]["influential_cases"][24])
inf_cases_24

,petal-width,.session_training_index,.influence_weight,petal-length,target,sepal-length,.session,sepal-width
0,1.5,22,0.129053,4.9,1,6.9,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.1
1,1.5,36,0.128127,4.7,1,6.7,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.1
2,1.8,77,0.126280,5.5,2,6.5,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.0
3,1.8,32,0.123981,5.5,2,6.4,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.1
4,1.8,47,0.123346,4.9,2,6.3,c2369bd4-238f-40e9-8259-ddb4253b3c67,2.7
5,1.5,17,0.123215,5.1,2,6.3,c2369bd4-238f-40e9-8259-ddb4253b3c67,2.8
6,1.8,108,0.123034,4.9,2,6.1,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.0
7,1.5,24,0.122964,4.6,1,6.5,c2369bd4-238f-40e9-8259-ddb4253b3c67,2.8


Compare that to the influential cases for case `29`, which is firmly in the center of the solo cluster.  Here we see a single target value.

In [7]:
inf_cases_29 = pd.DataFrame(reaction["details"]["influential_cases"][29])
inf_cases_29

,petal-width,.session_training_index,.influence_weight,petal-length,target,sepal-length,.session,sepal-width
0,0.5,115,0.127613,1.7,0,5.1,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.3
1,0.3,27,0.126686,1.4,0,5.1,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.5
2,0.3,56,0.125578,1.3,0,5.0,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.5
3,0.2,89,0.125353,1.5,0,5.0,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.4
4,0.4,55,0.124283,1.5,0,5.4,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.4
5,0.2,14,0.123788,1.4,0,5.0,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.3
6,0.2,43,0.123433,1.4,0,5.1,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.5
7,0.2,101,0.123265,1.5,0,5.2,c2369bd4-238f-40e9-8259-ddb4253b3c67,3.5


With the influential cases we can derive additional insights, such as identifying anomalous cases from within the influential cases using `similarity_conviction`.  
This can help to identify data that are making predictions more noisy or identify what type of data should be collected to improve predictive power in the future.

In [8]:
inf_case_indices = inf_cases_24[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal-length", "sepal-width", "petal-length", "petal-width", "target", "similarity_conviction"]
)

anom_df.sort_values(by="similarity_conviction")

,sepal-length,sepal-width,petal-length,petal-width,target,similarity_conviction
0,6.9,3.1,4.9,1.5,1,0.984911
5,6.3,2.8,5.1,1.5,2,0.998538
7,6.5,2.8,4.6,1.5,1,1.009214
3,6.4,3.1,5.5,1.8,2,1.010983
2,6.5,3.0,5.5,1.8,2,1.013636
1,6.7,3.1,4.7,1.5,1,1.018396
6,6.1,3.0,4.9,1.8,2,1.030155
4,6.3,2.7,4.9,1.8,2,1.033912


In [9]:
inf_case_indices = inf_cases_29[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal-length", "sepal-width", "petal-length", "petal-width", "target", "similarity_conviction"],
)

anom_df.sort_values(by="similarity_conviction")

,sepal-length,sepal-width,petal-length,petal-width,target,similarity_conviction
0,5.1,3.3,1.7,0.5,0,0.953567
4,5.4,3.4,1.5,0.4,0,0.986969
2,5.0,3.5,1.3,0.3,0,1.007771
5,5.0,3.3,1.4,0.2,0,1.010542
3,5.0,3.4,1.5,0.2,0,1.013599
1,5.1,3.5,1.4,0.3,0,1.014387
6,5.1,3.5,1.4,0.2,0,1.018135
7,5.2,3.5,1.5,0.2,0,1.021248


### Insight 2: Which Features Contributed?

In addition to providing attribution to cases, Howso also provides robust feature contributions to explain which features contributed to each prediction.
When inspecting the feature importances for case `24`,  we see that `petal-width` and `petal-length` provided the majority of the contribution to the 
prediction, whereas for case `29` `petal-width` and `petal-length` only have slightly higher contributions than the other features.  This could indicate 
that different features are more important for predicting different classes within this dataset and that focusing on those features would be prudent.

In [10]:
fcs_24 = pd.DataFrame(reaction["details"]["feature_contributions"][24:25], index=[24])
fcs_29 = pd.DataFrame(reaction["details"]["feature_contributions"][29:30], index=[29])

display(fcs_24)
display(fcs_29)


,petal-width,petal-length,sepal-length,sepal-width
24,0.353597,0.22389,0.099016,0.049315


,petal-width,petal-length,sepal-length,sepal-width
29,0.193158,0.194967,0.125331,0.108633


### Insight 3: How Certain is the Prediction?

[Residuals](https://docs.howso.com/user_guide/basic_capabilities/residuals.html) can characterize the uncertainty of the data around the prediction.  This will tell us which features are hard to predict in the region of the data around each case we're predicting.
If a prediction is less accurate than expected, this can explain which features were noisy and may have contributed to the problem.

In [11]:
pd.DataFrame(reaction["details"]["feature_residuals"])

,petal-width,petal-length,target,sepal-length,sepal-width
0,0.303407,0.321109,0.135434,0.250106,0.152110
1,0.201253,0.352991,0.317824,0.203021,0.222641
2,0.187110,0.361751,0.323383,0.339382,0.199041
3,0.412683,0.471883,0.277863,0.331419,0.139323
4,0.219836,0.274895,0.225341,0.368469,0.252307
5,0.181884,0.554572,0.055472,0.332925,0.291596
6,0.215172,0.313651,0.192183,0.382037,0.220704
7,0.202946,0.298681,0.178190,0.335324,0.311656
8,0.106969,0.197829,0.268997,0.330842,0.198229
9,0.110712,0.163008,0.046459,0.317863,0.164311


We can also use [`residual conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#prediction-residual-conviction) to determine which features are uncertain in a scale-invariant manner,
which can be useful if you wish to compare different features of different scales against each other.

In [12]:
pd.DataFrame(reaction["details"]["local_case_feature_residual_convictions"])

,petal-width,petal-length,target,sepal-length,sepal-width
0,1.170071,1.931268,1.759374,1.580108,0.602382
1,2.202485,1.575183,0.898450,1.803481,1.029518
2,2.647763,1.446682,1.500161,0.838021,1.748231
3,1.001863,1.369058,0.472099,1.036841,0.425108
4,1.750091,0.787880,0.774590,1.508964,1.004640
5,1.007053,1.757039,1.090460,3.315311,1.696239
6,0.771271,0.453613,0.550969,0.467964,1.423346
7,0.993329,0.341920,0.423613,0.665097,1.192785
8,1.193742,0.652301,0.951319,1.600084,0.726766
9,1.215589,1.265492,1.328492,1.117219,1.137440


### Insight 4: How Anomalous are the Predicted Cases?

By getting the [`similarity conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#similarity-conviction) of the cases that we predict, we can determine which of them are anomalous relative to the trained data.
This could help to highlight cases that are unusually difficult or easy to predict, as well as discover potentially malicious or poisoned cases.

In [13]:
pprint(reaction["details"]["similarity_conviction"], compact=True)

[1.003894609677443, 1.020494859879364, 1.0477951779250738, 0.9838676801659485,
 1.0197365021684897, 1.0260969129904407, 0.9200538474774768, 0.9118563680231693,
 1.013381731767785, 1.003633842730834, 0.9628174120337305, 1.0023765504180948,
 0.9978842397243503, 1.009395384525202, 1.0080168714340827, 1.0170541036407086,
 0.9877560804529695, 0.9765352704670582, 0.96677263878207, 1.0128115565942952,
 1.002183541948585, 0.9889392882651399, 1.010790614638105, 1.014675332147475,
 0.9957948406751697, 0.9544512319635702, 1.0035601461352501, 1.0148016148109456,
 1.0146375968262136, 0.9993000375482713]
